based on [this comment][1]

  [1]: https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/forums/t/6650/share-your-approach?forumMessageId=36434#post36434

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.

In [39]:
import pandas as pd
import IPython.display
from six.moves import cPickle as pickle

def maybe_pickle(file_name, load_dataset, force=False):
    pickle_file_name = "pickle/" + file_name + ".pickle"
    import os
    if not os.path.exists("pickle"):
        os.makedirs("pickle")
        
    if os.path.exists(file_name) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping pickling.' % file_name)
    else:
        print('Pickling %s.' % file_name)
        dataset = load_dataset(None)
        try:
            with open(file_name, 'wb') as f:
                pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', file_name, ':', e)
    
    return pickle_file_name

def load_data(file_name):
    original_file_path = "../input/" + file_name + ".csv"
    pickle_file_name = maybe_pickle(file_name, lambda x: pd.read_csv(original_file_path))
    
    with open(pickle_file_name, 'rb') as f:
        return pickle.load(f)

In [40]:
biology = load_data("biology")
cooking = load_data("cooking")
biology = load_data("biology")
biology = load_data("biology")
biology = load_data("biology")
biology = load_data("biology")
biology = load_data("biology")
biology = load_data("biology")


cooking = pd.read_csv("../input/cooking.csv")
crypto = pd.read_csv("../input/crypto.csv")
diy = pd.read_csv("../input/diy.csv")
robotics = pd.read_csv("../input/robotics.csv")
sample_submission = pd.read_csv("../input/sample_submission.csv")
test = pd.read_csv("../input/test.csv")
travel = pd.read_csv("../input/travel.csv")

pickle/biology.pickle already present - Skipping pickling.
pickle/cooking.pickle already present - Skipping pickling.
pickle/biology.pickle already present - Skipping pickling.
pickle/biology.pickle already present - Skipping pickling.
pickle/biology.pickle already present - Skipping pickling.
pickle/biology.pickle already present - Skipping pickling.
pickle/biology.pickle already present - Skipping pickling.
pickle/biology.pickle already present - Skipping pickling.


In [3]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
stemmer = SnowballStemmer("english")
wordnet_lemmatizer = WordNetLemmatizer()
punctuation_trans_table = str.maketrans({key: None for key in string.punctuation})
html_tag_regex = re.compile('<.*?>')

print(biology.head())

def cleaning_text(text):
    # TODO remove code fragment
    # TODO remove url
    # TODO convert to lowercase
    # TODO add meta features from original text
    ## length of the raw text in chars
    ## number of code segments
    ## number of 'a href' tags
    ## number of times 'http' occurs (count urls)
    ## number of times 'grater sign' occurs (count html tags)
    # TODO add meta features from cleaned text
    ## number of words(tokens) in the clean text
    ## length of the clean text in chars
    # TODO feature scaling(0-1 range) with min-max
    
    # remove html tags
    text = re.sub(html_tag_regex, '', text)
    # remove \r, \n
    text = text.replace('\n', ' ').replace('\r', '')
    # remove Punctuations
    text = text.translate(punctuation_trans_table)
    # split
    words = word_tokenize(text)
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # lemmatizing, stemming
    words = [stemmer.stem(wordnet_lemmatizer.lemmatize(word)) for word in words]
    # join
    text = ' '.join(words)
    return text


def cleaning(row):
    row['title'] = cleaning_text(row['title'])
    row['content'] = cleaning_text(row['content'])
    return row

# TODO remove duplicates
# TODO union tags for duplicates
cleaned_df = biology.apply(cleaning, axis=1)
print(cleaned_df.head())

   id                                              title  \
0   1  What is the criticality of the ribosome bindin...   
1   2  How is RNAse contamination in RNA based experi...   
2   3      Are lymphocyte sizes clustered in two groups?   
3   4  How long does antibiotic-dosed LB maintain goo...   
4   5        Is exon order always preserved in splicing?   

                                             content  \
0  <p>In prokaryotic translation, how critical fo...   
1  <p>Does anyone have any suggestions to prevent...   
2  <p>Tortora writes in <em>Principles of Anatomy...   
3  <p>Various people in our lab will prepare a li...   
4  <p>Are there any cases in which the splicing m...   

                                                tags  
0  ribosome binding-sites translation synthetic-b...  
1                                   rna biochemistry  
2                 immunology cell-biology hematology  
3                                       cell-culture  
4            splicing mrna 

In [4]:
print(len(cleaned_df))

13196


In [5]:
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stop_words)
print(cleaned_df[['content']])
trans_word_data = vectorizer.fit_transform(cleaned_df['content'].tolist())
print(trans_word_data)

                                                 content
0      in prokaryot translat critic effici translat l...
1      doe anyon suggest prevent rnase contamin work ...
2      tortora write principl anatomi physiolog lymph...
3      various peopl lab prepar liter lb add kanamyci...
4      are case splice machineri construct mrna exon ...
5      im interest sequenc analyz bound dna minim amo...
6      im look resourc inform format dendrit spine sy...
7      i ship 10 µl vector miniprep collabor 15 ml ep...
8      i notic within exampl experi class differ repo...
9      accord endosymbiont theori mitochondrion chlor...
10     in highthroughput experi cell cultur treat sta...
11     has anyon tri chew backann vitro dna assembl m...
12     the whole question what optim frame size secon...
13     we know pop scienc psycholog state effect immu...
14     there bioinformat consider show signific corre...
15     i recent download gene annot homo sapien ensem...
16     what strain design flu m

In [1]:
# TODO extract most common tags
tags_list = cleaned_df['tags'].str.split(pat=' ').tolist()
total_tags = pd.Series([item for sublist in tags_list for item in sublist])
print(len(total_tags))
print(total_tags.value_counts())

NameError: name 'cleaned_df' is not defined

In [7]:
# TODO train SGD classifier with one-vs-rest approach
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="modified_huber")